In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
import os

In [4]:
sess = tf.InteractiveSession()

In [5]:
PIXEL_DEPTH = 255
# IMAGE_SIZE = 20 # for chars74k
IMAGE_SIZE = 28 # for MNIST
NUM_CHANNELS = 1
DATASETDIR = 'chars74k-lite'

In [6]:
def fileListFunc(filePathList):
    fileList = []
    for filePath in filePathList:
        for top, dirs, nondirs in os.walk(filePath):
            print(top, dirs, nondirs)
            for item in nondirs:
                fileList.append(os.path.join(top, item))
    return fileList

In [7]:
def extract_data(filename):
    image_raw_data = tf.gfile.FastGFile(filename, 'rb').read()
    img_data = tf.image.decode_jpeg(image_raw_data)
    data = img_data.eval()
    data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH
    return data

In [8]:
character_mapping = {
    0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9,
    10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G',
    17: 'H', 18: 'I', 19: 'J', 20: 'K', 21: 'L', 22: 'M', 23: 'N',
    24: 'O', 25: 'P', 26: 'Q', 27: 'R', 28: 'S', 29: 'T', 30: 'U',
    31: 'V', 32: 'W', 33: 'X', 34: 'Y', 35: 'Z', 36: 'a', 37: 'b',
    38: 'c', 39: 'd', 40: 'e', 41: 'f', 42: 'g', 43: 'h', 44: 'i',
    45: 'j', 46: 'k', 47: 'l', 48: 'm', 49: 'n', 50: 'o', 51: 'p',
    52: 'q', 53: 'r', 54: 's', 55: 't', 56: 'u', 57: 'v', 58: 'w',
    59: 'x', 60: 'y', 61: 'z'
}

In [9]:
print(ord('a')-61)

36


- 来自于MNIST的原始数据集

In [10]:
import gzip
import numpy
def extract_data(filename, num_images):
    NUM_CHANNELS = 1
    PIXEL_DEPTH = 255
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    print("Extracting", filename)
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(IMAGE_SIZE * IMAGE_SIZE * num_images * NUM_CHANNELS)
        data = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.float32)
        data = (data - (PIXEL_DEPTH / 2.0)) / PIXEL_DEPTH
        data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
        return data
def extract_labels(filename, num_images):
    """Extract the labels into a vector of int64 label IDs."""
    print("Extracting", filename)
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = numpy.frombuffer(buf, dtype=numpy.uint8).astype(numpy.int64)
    return labels
train_data = extract_data("input/train-images-idx3-ubyte.gz", 60000)
train_labels = extract_labels("input/train-labels-idx1-ubyte.gz", 60000)

Extracting input/train-images-idx3-ubyte.gz
Extracting input/train-labels-idx1-ubyte.gz


- 来自于chars74k的字母图片数据集

In [10]:
train_data = []
train_labels = []
for each in os.listdir(DATASETDIR):
    dirname = DATASETDIR+'/'+each
    if os.path.isdir(dirname):
        print(each,end=',')
        for filename in os.listdir(dirname):
            train_data.append(extract_data(dirname+'/'+filename))
            train_labels.append(ord(each)-61)

a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,

In [11]:
len(train_labels)

60000

In [12]:
len(train_data), len(train_data[0])

(60000, 28)

In [13]:
train_data_node = tf.placeholder(
    tf.float32,
    shape=(None, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS),
    name="image_input",
)

In [14]:
train_labels_node = tf.placeholder(tf.int64, shape=(None,))

In [15]:
train_size = np.array(train_labels).shape[0]

In [16]:
eval_data = tf.placeholder(
    tf.float32, shape=(None, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
)

In [17]:
def data_type():
    return tf.float32

In [18]:
SEED = 66478
# NUM_LABELS = 26 # for chars74k dataset
NUM_LABELS = 10 # for mnist dataset

In [19]:
def inference(data, train=False):
    # The variables below hold all the trainable weights. They are passed an
    # initial value which will be assigned when we call:
    # {tf.global_variables_initializer().run()}
    conv1_weights = tf.Variable(
        tf.truncated_normal(
            [5, 5, NUM_CHANNELS, 32],  # 5x5 filter, depth 32.
            stddev=0.1,
            seed=SEED,
            dtype=data_type(),
        )
    )
    conv1_biases = tf.Variable(tf.zeros([32], dtype=data_type()))
    conv2_weights = tf.Variable(
        tf.truncated_normal([5, 5, 32, 64], stddev=0.1, seed=SEED, dtype=data_type())
    )
    conv2_biases = tf.Variable(tf.constant(0.1, shape=[64], dtype=data_type()))
    fc1_weights = tf.Variable(  # fully connected, depth 512.
        tf.truncated_normal(
            [IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * 64, 512],
            stddev=0.1,
            seed=SEED,
            dtype=data_type(),
        )
    )
    fc1_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=data_type()))
    fc2_weights = tf.Variable(
        tf.truncated_normal([512, NUM_LABELS], stddev=0.1, seed=SEED, dtype=data_type())
    )
    fc2_biases = tf.Variable(tf.constant(0.1, shape=[NUM_LABELS], dtype=data_type()))
    layers = [fc1_weights, fc1_biases, fc2_weights, fc2_biases]
    """The Model definition."""
    # 2D convolution, with 'SAME' padding (i.e. the output feature map has
    # the same size as the input). Note that {strides} is a 4D array whose
    # shape matches the data layout: [image index, y, x, depth].
    conv = tf.nn.conv2d(data, conv1_weights, strides=[1, 1, 1, 1], padding="SAME")
    # Bias and rectified linear non-linearity.
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    # Max pooling. The kernel size spec {ksize} also follows the layout of
    # the data. Here we have a pooling window of 2, and a stride of 2.
    pool = tf.nn.max_pool(
        relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME"
    )
    conv = tf.nn.conv2d(pool, conv2_weights, strides=[1, 1, 1, 1], padding="SAME")
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
    pool = tf.nn.max_pool(
        relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME"
    )
    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool.get_shape().as_list()
    reshape = tf.reshape(
        pool, [tf.shape(pool)[0], pool_shape[1] * pool_shape[2] * pool_shape[3]]
    )
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    # Add a 50% dropout during training only. Dropout also scales
    # activations such that no rescaling is needed at evaluation time.
    if train:
        hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
    return tf.matmul(hidden, fc2_weights) + fc2_biases, layers

In [20]:
logits, layers = inference(train_data_node, True)

In [21]:
train_labels_node

<tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>

In [22]:
loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=train_labels_node, logits=logits
    )
)

In [23]:
predict_op = tf.argmax(logits, 1, name="predict_op")

In [24]:
regularizers = 0
for each in layers:
    regularizers += tf.nn.l2_loss(each)

In [25]:
loss += 5e-4 * regularizers

In [26]:
BATCH_SIZE = 64

开始训练

In [27]:
# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0, dtype=data_type())
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    0.01,  # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,  # Decay step.
    0.95,  # Decay rate.
    staircase=True,
)
# Use simple momentum for the optimization.
optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(
    loss, global_step=batch
)

# Predictions for the current training minibatch.
train_prediction = tf.nn.softmax(logits)

# Predictions for the test and validation, which we'll compute less often.
eval_prediction = tf.nn.softmax(inference(eval_data)[0])

In [28]:
EVAL_BATCH_SIZE = 64
EVAL_FREQUENCY = 100  # Number of steps between evaluations.

In [29]:
# Small utility function to evaluate a dataset by feeding batches of data to
# {eval_data} and pulling the results from {eval_predictions}.
# Saves memory and enables this to run on smaller GPUs.
def eval_in_batches(data, sess):
    """Get all predictions for a dataset by running it in small batches."""
    size = data.shape[0]
    if size < EVAL_BATCH_SIZE:
        raise ValueError("batch size for evals larger than dataset: %d" % size)
    predictions = numpy.ndarray(shape=(size, NUM_LABELS), dtype=numpy.float32)
    for begin in xrange(0, size, EVAL_BATCH_SIZE):
        end = begin + EVAL_BATCH_SIZE
        if end <= size:
            predictions[begin:end, :] = sess.run(
                eval_prediction, feed_dict={eval_data: data[begin:end, ...]}
            )
        else:
            batch_predictions = sess.run(
                eval_prediction, feed_dict={eval_data: data[-EVAL_BATCH_SIZE:, ...]}
            )
            predictions[begin:, :] = batch_predictions[begin - size :, :]
    return predictions

In [30]:
EXPORT_DIR = 'export'
MODEL_DIR = 'output'
NUM_EPOCHS = 10

In [31]:
import time
import shutil
from six.moves import xrange

In [32]:
### Change original code
# Create a saver for writing training checkpoints.
saver = tf.train.Saver()
# Create a builder for writing saved model for serving.
if os.path.isdir(EXPORT_DIR):
    shutil.rmtree(EXPORT_DIR)
builder = tf.saved_model.builder.SavedModelBuilder(EXPORT_DIR)

# Create a local session to run the training.
start_time = time.time()

以下开始跑Session

In [33]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)

In [34]:
# Run all the initializers to prepare the trainable parameters.
tf.global_variables_initializer().run()
### Change original code
# Save checkpoint when training
ckpt = tf.train.get_checkpoint_state(MODEL_DIR)
if ckpt and ckpt.model_checkpoint_path:
    print("Load from " + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)

### Change original code
# Create summary, logs will be saved, which can display in Tensorboard
tf.summary.scalar("loss", loss)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(
    os.path.join(MODEL_DIR, "log"), sess.graph
)

print("Initialized!")

Load from output\model.ckpt-8000
INFO:tensorflow:Restoring parameters from output\model.ckpt-8000
Initialized!


In [35]:
xrange(int(NUM_EPOCHS * train_size) // BATCH_SIZE)

range(0, 9375)

In [36]:
step = 0
offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
batch_data = train_data[offset : (offset + BATCH_SIZE), ...]
batch_labels = train_labels[offset : (offset + BATCH_SIZE)]

In [37]:
batch_labels.dtype

dtype('int64')

In [38]:
feed_dict = {train_data_node: batch_data, train_labels_node: batch_labels}

In [39]:
optimizer.run(feed_dict=feed_dict)

In [40]:
def error_rate(predictions, labels):
    """Return the error rate based on dense predictions and sparse labels."""
    return 100.0 - (
        100.0 * numpy.sum(numpy.argmax(predictions, 1) == labels) / predictions.shape[0]
    )

In [41]:
import numpy
import sys

In [42]:
VALIDATION_SIZE = 5000
validation_data = train_data[:VALIDATION_SIZE, ...]
validation_labels = train_labels[:VALIDATION_SIZE]

In [43]:
# Loop through training steps.
for step in xrange(int(NUM_EPOCHS * train_size) // BATCH_SIZE):
    # Compute the offset of the current minibatch in the data.
    # Note that we could use better randomization across epochs.
    offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
    batch_data = train_data[offset : (offset + BATCH_SIZE), ...]
    batch_labels = train_labels[offset : (offset + BATCH_SIZE)]
    # This dictionary maps the batch data (as a numpy array) to the
    # node in the graph it should be fed to.
    feed_dict = {train_data_node: batch_data, train_labels_node: batch_labels}
    # Run the optimizer to update weights.
    # sess.run(optimizer, feed_dict=feed_dict)
    optimizer.run(feed_dict=feed_dict)
    # print some extra information once reach the evaluation frequency
    if step % EVAL_FREQUENCY == 0:
        # fetch some extra nodes' data
        ### Change original code
        # Add summary
        summary, l, lr, predictions = sess.run(
            [merged, loss, learning_rate, train_prediction], feed_dict=feed_dict
        )
        writer.add_summary(summary, step)
        elapsed_time = time.time() - start_time
        start_time = time.time()
        ### Change original code
        # save model
        if step % (EVAL_FREQUENCY * 10) == 0:
            saver.save(
                sess,
                os.path.join(MODEL_DIR, "model.ckpt"),
                global_step=step,
            )
        print(
            "Step %d (epoch %.2f), %.1f ms"
            % (
                step,
                float(step) * BATCH_SIZE / train_size,
                1000 * elapsed_time / EVAL_FREQUENCY,
            )
        )
        print("Minibatch loss: %.3f, learning rate: %.6f" % (l, lr))
        print("Minibatch error: %.1f%%" % error_rate(predictions, batch_labels))
        print(
            "Validation error: %.1f%%"
            % error_rate(
                eval_in_batches(validation_data, sess), validation_labels
            )
        )
        sys.stdout.flush()

Step 0 (epoch 0.00), 176.2 ms
Minibatch loss: 0.731, learning rate: 0.001748
Minibatch error: 0.0%
Validation error: 91.3%
Step 100 (epoch 0.11), 6.2 ms
Minibatch loss: 0.731, learning rate: 0.001748
Minibatch error: 0.0%
Validation error: 91.3%
Step 200 (epoch 0.21), 5.2 ms
Minibatch loss: 0.762, learning rate: 0.001748
Minibatch error: 1.6%
Validation error: 91.3%
Step 300 (epoch 0.32), 5.0 ms
Minibatch loss: 0.728, learning rate: 0.001748
Minibatch error: 0.0%
Validation error: 91.3%
Step 400 (epoch 0.43), 5.0 ms
Minibatch loss: 0.726, learning rate: 0.001748
Minibatch error: 0.0%
Validation error: 91.3%
Step 500 (epoch 0.53), 5.2 ms
Minibatch loss: 0.729, learning rate: 0.001748
Minibatch error: 0.0%
Validation error: 91.3%
Step 600 (epoch 0.64), 5.1 ms
Minibatch loss: 0.731, learning rate: 0.001748
Minibatch error: 0.0%
Validation error: 91.3%
Step 700 (epoch 0.75), 5.1 ms
Minibatch loss: 0.729, learning rate: 0.001748
Minibatch error: 0.0%
Validation error: 91.3%
Step 800 (epoch 

In [ ]:
### Change original code
# Save model
inputs = {tf.saved_model.signature_constants.PREDICT_INPUTS: train_data_node}
outputs = {tf.saved_model.signature_constants.PREDICT_OUTPUTS: predict_op}
serving_signatures = {
    "Infer": tf.saved_model.signature_def_utils.predict_signature_def(  # tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
        inputs, outputs
    )
}
builder.add_meta_graph_and_variables(
    sess,
    [tf.saved_model.tag_constants.SERVING],
    signature_def_map=serving_signatures,
    assets_collection=tf.get_collection(tf.GraphKeys.ASSET_FILEPATHS),
    clear_devices=True,
)
builder.save()

# Finally print the result!
test_error = error_rate(eval_in_batches(test_data, sess), test_labels)
print("Test error: %.1f%%" % test_error)
if config.FLAGS.self_test:
    print("test_error", test_error)
    assert test_error == 0.0, "expected 0.0 test_error, got %.2f" % (
        test_error,
    )